In [ ]:
# import matplotlib
from sklearn.preprocessing import LabelBinarizer
# from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
# from keras.layers.core import Dense
import utils_paths # 用於操作路徑
import numpy as np
import random
import cv2
import os
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

train_data = [] #x train
train_label = [] # y train
test_data = [] # x test
test_label = [] # y test

# 取路徑方便讀取
train_path = sorted(list(utils_paths.list_images('./CIFAR10/train')))
test_path = sorted(list(utils_paths.list_images('./CIFAR10/test')))

random.seed(42)
# random.shuffle(imagePaths)
for imagePath in train_path:
    image = cv2.imread(imagePath) # 存 train圖
    # image = cv2.resize(image, (32, 32)).flatten() # 拉成一維(3072,)
    train_data.append(image) # 存train label
    
    label = imagePath.split(os.path.sep)[-2]
    train_label.append(label) # 存LABEL

for imagePath in test_path:
    image = cv2.imread(imagePath)
    test_data.append(image) # 存 test圖
    label = imagePath.split(os.path.sep)[-2]
    test_label.append(label) # 存test label

x_train, y_train, x_test, y_test = train_data, train_label, test_data, test_label
# x y = list

# 圖 scale and np
x_train = np.array(x_train, dtype="float") / 255.0
x_test = np.array(x_test, dtype="float") / 255.0
y_train = np.array(y_train)
y_test = np.array(y_test)


#trans label to number
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

# print(x_train.shape) #(50000, 32, 32, 3)
# print(y_train.shape) #(50000, 10)
# print(x_test.shape) #(10000, 32, 32, 3)
# print(y_test.shape) #(10000, 10)

model=Sequential()
model.add(Conv2D(input_shape = (32,32,3), filters=16, kernel_size=(5,5), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=36, kernel_size=(5, 5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

es = [tf.keras.callbacks.EarlyStopping(monitor='accuracy', mode='max', verbose=1, 
                                               patience=10, min_delta=0.001)]

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=50, batch_size=120, verbose=1, callbacks=es)

pred = model.predict(x_test)
pred_oh = np.zeros_like(pred) # pred one hot
pred_oh[np.arange(len(pred)), pred.argmax(1)] = 1

score = model.evaluate(x_test, y_test)
print(score)
cm = confusion_matrix(y_test, pred)
cr = classification_report(y_test, pred_oh, digits=4)
print(cr)